In [1]:
import sys
sys.path.append('../')
import warnings
warnings.filterwarnings('ignore')

### Testing `PredictionService` API

In [2]:
from apis import *

In [3]:
model_spec = ModelSpec(name='mnist', version=2, signature_name='predict_images')

In [4]:
model_spec

name: "mnist"
version {
  value: 2
}
signature_name: "predict_images"

In [5]:
import mnist_input_data

In [6]:
test_data_set = mnist_input_data.read_data_sets('/tmp').test

W0825 22:32:29.505696 139853570537280 deprecation.py:323] From <ipython-input-6-17e66ce9737f>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0825 22:32:29.506736 139853570537280 deprecation.py:323] From /home/jagan/.virtualenvs/tfs-client-env/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0825 22:32:29.507449 139853570537280 deprecation.py:323] From /home/jagan/.virtualenvs/tfs-client-env/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is depre

Extracting /tmp/train-images-idx3-ubyte.gz


W0825 22:32:29.747982 139853570537280 deprecation.py:323] From /home/jagan/.virtualenvs/tfs-client-env/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0825 22:32:29.795783 139853570537280 deprecation.py:323] From /home/jagan/.virtualenvs/tfs-client-env/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting /tmp/train-labels-idx1-ubyte.gz
Extracting /tmp/t10k-images-idx3-ubyte.gz
Extracting /tmp/t10k-labels-idx1-ubyte.gz


In [7]:
# Test
predict_request = PredictRequest(model_spec=model_spec)
get_metadata_request = GetModelMetadataRequest(model_spec, 'signature_def')
prediction_service = PredictionService('0.0.0.0:8500')

In [8]:
predict_output_list = []
actual_output_list = []
metadata_output_list = []
for _ in range(2):
    image, label = test_data_set.next_batch(1)
    predict_request.inputs = {'images' : image[0].reshape(1,-1)}
    actual_output_list.append(label)
    predict_output_list.append(prediction_service.predict(predict_request).parse_outputs())
    metadata_output_list.append(prediction_service.get_model_metadata(get_metadata_request)) # not parsed :(

In [9]:
import numpy as np

#### Truth vs Prediction

In [10]:
for truth, pred in zip(actual_output_list, predict_output_list):
    print(truth, np.argmax(pred['scores'], axis=-1))

[1] [1]
[6] [6]


In [11]:
metadata_output_list

[model_spec {
   name: "mnist"
   version {
     value: 2
   }
 }
 metadata {
   key: "signature_def"
   value {
     type_url: "type.googleapis.com/tensorflow.serving.SignatureDefMap"
     value: "\n\305\001\n\017serving_default\022\261\001\n\036\n\006inputs\022\024\n\014tf_example:0\020\007\032\002\030\001\022)\n\006scores\022\037\n\010TopKV2:0\020\001\032\021\022\013\010\377\377\377\377\377\377\377\377\377\001\022\002\010\n\022G\n\007classes\022<\n%hash_table_Lookup/LookupTableFindV2:0\020\007\032\021\022\013\010\377\377\377\377\377\377\377\377\377\001\022\002\010\n\032\033tensorflow/serving/classify\n{\n\016predict_images\022i\n%\n\006images\022\033\n\003x:0\020\001\032\022\022\013\010\377\377\377\377\377\377\377\377\377\001\022\003\010\220\006\022$\n\006scores\022\032\n\003y:0\020\001\032\021\022\013\010\377\377\377\377\377\377\377\377\377\001\022\002\010\n\032\032tensorflow/serving/predict"
   }
 }, model_spec {
   name: "mnist"
   version {
     value: 2
   }
 }
 metadata {
   k

#### TODO: Parse `signature_def`

### Testing `ModelService` API

In [12]:
from config import *
from sources import *

In [13]:
version_policy_a = ServableVersionPolicy(specific=1)
version_policy_b = ServableVersionPolicy(_all=1)

In [14]:
model_config_a = ModelConfig(name='mnist', 
                             base_path='/models/mnist', 
                             model_platform='tensorflow', 
                             model_version_policy=version_policy_a)

In [15]:
model_config_b = ModelConfig(name='mnist', 
                             base_path='/models/mnist', 
                             model_platform='tensorflow', 
                             model_version_policy=version_policy_b)

### Running `A`, i.e. Load version #1

#### Reload

In [17]:
model_config_list = ModelConfigList(config=[model_config_a])
model_server_config = ModelServerConfig(model_config_list=model_config_list)

In [18]:
reload_config_request = ReloadConfigRequest(config=model_server_config)
model_service = ModelService('0.0.0.0:8500')
reload_config_response = model_service.reload_config(reload_config_request)

In [19]:
reload_config_response # if successful returns an empty status

status {
}

#### Get Status

In [20]:
model_spec = ModelSpec(name='mnist', signature_name='predict_images')

In [22]:
get_model_status_request = GetModelStatusRequest(model_spec=model_spec)

In [23]:
get_model_status_response = model_service.get_model_status(get_model_status_request)

In [25]:
get_model_status_response

model_version_status {
  version: 2
  state: END
  status {
  }
}
model_version_status {
  version: 1
  state: AVAILABLE
  status {
  }
}

Version 1 is running and 2 has been taken down.

### Running `B`, i.e. Load all versions

#### Reload

In [26]:
model_config_list = ModelConfigList(config=[model_config_b])
model_server_config = ModelServerConfig(model_config_list=model_config_list)

In [27]:
reload_config_request = ReloadConfigRequest(config=model_server_config)
model_service = ModelService('0.0.0.0:8500')
reload_config_response = model_service.reload_config(reload_config_request)

In [28]:
reload_config_response # if successful returns an empty status

status {
}

#### Get Status

In [29]:
model_spec = ModelSpec(name='mnist', signature_name='predict_images')

In [30]:
get_model_status_request = GetModelStatusRequest(model_spec=model_spec)

In [31]:
get_model_status_response = model_service.get_model_status(get_model_status_request)

In [32]:
get_model_status_response

model_version_status {
  version: 2
  state: AVAILABLE
  status {
  }
}
model_version_status {
  version: 1
  state: AVAILABLE
  status {
  }
}

Both the versions are running.

### Running config `C`, i.e. Unload all versions

#### Reload

In [33]:
model_server_config = ModelServerConfig()
# it's important you initialize the config as empty, otherwise you'll get a gRPC error
model_server_config.model_config_list.config = []  # sending an empty model_server_config

In [34]:
reload_config_request = ReloadConfigRequest(config=model_server_config)
model_service = ModelService('0.0.0.0:8500')
reload_config_response = model_service.reload_config(reload_config_request)

In [35]:
reload_config_response # if successful returns an empty status

status {
}

#### Get Status

In [36]:
model_spec = ModelSpec(name='mnist', signature_name='predict_images')

In [37]:
get_model_status_request = GetModelStatusRequest(model_spec=model_spec)

In [38]:
get_model_status_response = model_service.get_model_status(get_model_status_request)

In [39]:
get_model_status_response

model_version_status {
  version: 2
  state: END
  status {
  }
}
model_version_status {
  version: 1
  state: END
  status {
  }
}

All versions have been taken down.